<a href="https://colab.research.google.com/github/kimdonggyu2008/hufs_hackerthon/blob/main/BBC_edit_5epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install wandb

In [3]:
!pip install keras.preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import string
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

#import wandb
import os

In [5]:
# os.environ['WANDB_API_KEY']='513a1f0c050fa7f60a76b5232e904d8df397082e'
# os.environ['WANDB_ENTITY']='article classification'

In [6]:
# !wandb login

#감정분석(필요?)

In [7]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 22.9 MB/s eta 0:00:00


In [8]:
!apt-get update -qq
!apt-get install -qq fonts-nanum

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 123621 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

In [10]:
#augmented는 500개로 맞춤
#augmented2는 1000개로 맞춤
df=pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train_augmented2.csv')

In [11]:
okt=Okt()

In [12]:
def preprocess_korean(text):
    # 특수문자 제거 및 소문자 변환
    text = re.sub(r'\W+', ' ', text)
    # 형태소 분석 후 명사만 추출
    tokens = okt.nouns(text)
    return ' '.join(tokens)

#카테고리를 번호로 치환

In [13]:
# 카테고리 목록과 매핑될 레이블 번호를 생성합니다.
categories = [
    "지역", "경제:부동산", "사회:사건_사고", "경제:반도체", "사회:사회일반", "사회:교육_시험",
    "정치:국회_정당", "사회:의료_건강", "경제:취업_창업", "스포츠:올림픽_아시안게임",
    "경제:산업_기업", "문화:전시_공연", "경제:자동차", "경제:경제일반", "사회:장애인",
    "스포츠:골프", "정치:선거", "경제:유통", "IT_과학:모바일", "사회:여성",
    "사회:노동_복지", "사회:환경", "경제:서비스_쇼핑", "경제:무역", "정치:행정_자치",
    "국제", "문화:방송_연예", "스포츠:축구", "경제:금융_재테크", "정치:청와대",
    "문화:출판", "IT_과학:IT_과학일반", "IT_과학:인터넷_SNS", "문화:미술_건축",
    "정치:정치일반", "IT_과학:과학", "문화:문화일반", "문화:학술_문화재", "문화:요리_여행",
    "경제:자원", "문화:종교", "IT_과학:콘텐츠", "사회:미디어", "사회:날씨",
    "스포츠:농구_배구", "문화:음악", "문화:생활", "IT_과학:보안", "스포츠:월드컵",
    "경제:증권_증시", "정치:북한", "정치:외교", "스포츠:스포츠일반", "문화:영화",
    "스포츠:야구", "경제:외환"
]

# DataFrame을 랜덤하게 섞습니다.
shuffled = df.reindex(np.random.permutation(df.index))

# 각 카테고리에 대해 데이터 샘플링
num_of_categories = 45000
sampled_data = []
for category in categories:
    sampled_data.append(shuffled[shuffled['분류'] == category][:num_of_categories])

# 샘플링된 데이터를 하나의 데이터프레임으로 합칩니다.
concated = pd.concat(sampled_data, ignore_index=True)

# 데이터프레임을 다시 섞습니다.
concated = concated.reindex(np.random.permutation(concated.index))

# LABEL 열 생성 (카테고리별 레이블 부여)
concated['라벨'] = concated['분류'].apply(lambda x: categories.index(x))

# One-hot 인코딩
labels = to_categorical(concated['라벨'], num_classes=len(categories))

# 카테고리 열 삭제
# if '분류' in concated.columns:
#     concated = concated.drop(['분류'], axis=1)

# print(concated['라벨'][:10])
# print(concated['분류'])
# print(labels[:10])


In [14]:
n_most_common_words = 20000
max_len = 150
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(concated['제목'].values)
sequences = tokenizer.texts_to_sequences(concated['제목'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 83611 unique tokens.


In [15]:
# train = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train.csv')

# # 각 분류별 데이터 갯수 출력
# category_counts = train['분류'].value_counts()

# # 출력
# print(category_counts)

#데이터 증강 및 저장(1번만)


In [16]:
# train = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train.csv')

# # 각 분류별 데이터 갯수 확인
# category_counts = train['분류'].value_counts()

# # 500개 이하의 데이터를 가진 분류 추출
# under_500_categories = category_counts[category_counts < 1000].index.tolist()

# # 500개 이하의 분류에 대해 데이터 증강
# for category in under_500_categories:
#     current_count = category_counts[category]
#     additional_samples_needed = 1000 - current_count

#     # 해당 분류의 데이터를 복제하여 증강
#     category_data = train[train['분류'] == category]
#     augmented_data = category_data.sample(n=additional_samples_needed, replace=True, random_state=42)

#     # 원래 데이터에 증강된 데이터 추가
#     train = pd.concat([train, augmented_data])

# # 증강 후 데이터 확인
# new_category_counts = train['분류'].value_counts()
# print(new_category_counts)

# # 증강된 데이터 저장 (선택 사항)
# train.to_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train_augmented2.csv', index=False)

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D
from tensorflow.keras.layers import Add
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

#모델


In [18]:
# 기본 설정
n_most_common_words = 80000  # 어휘 수 80,000개로 설정
emb_dim = 128  # 임베딩 벡터의 차원 설정
max_len = X.shape[1]  # 시퀀스의 길이 (기사 제목의 길이)
num_heads = 8  # Multi-head attention에서 헤드의 개수
ff_dim = 128  # Feed Forward Network의 출력 차원
num_classes = 56  # 최종 클래스의 개수

# 최적화된 하이퍼파라미터
learning_rate = 0.00005        # 학습률 (조정 가능)
batch_size = 128             # 배치 크기
epochs = 20                  # 최대 에포크 수
dropout_rate = 0.8           # 드롭아웃 비율
l2_lambda = 0.001             # L2 정규화 값
patience = 7                  # 조기 종료를 위한 patience
min_delta = 0.001             # 조기 종료를 위한 최소 손실 감소량
validation_split = 0.2        # 검증 데이터 비율


In [19]:
emb_dim = 128
# 입력층
input_layer = Input(shape=(max_len,))

# Embedding 레이어
embedding_layer = Embedding(input_dim=n_most_common_words, output_dim=emb_dim, input_length=max_len)(input_layer)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [20]:

# 트랜스포머 블록 정의
def transformer_block(x, head_size, num_heads, ff_dim, dropout=0.3):
    # Multi-Head Attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    attention_output = Dropout(dropout)(attention_output)

    # Add & Normalize
    attention_output = Add()([x, attention_output])
    attention_output = LayerNormalization()(attention_output)

    # Feed Forward Network
    ffn_output = Dense(ff_dim, activation='relu',kernel_regularizer=l2(0.001))(attention_output)
    ffn_output = Dropout(dropout)(ffn_output)
    ffn_output = Dense(emb_dim)(ffn_output)

    # Add & Normalize
    ffn_output = Add()([attention_output, ffn_output])
    ffn_output = LayerNormalization()(ffn_output)

    return ffn_output


In [21]:
# 모델 빌드 함수
def build_model(n_most_common_words, emb_dim, max_len, num_heads, ff_dim, num_classes, dropout_rate):
    # 입력층
    input_layer = Input(shape=(max_len,))

    # Embedding 레이어
    embedding_layer = Embedding(input_dim=n_most_common_words, output_dim=emb_dim, input_length=max_len)(input_layer)

    # 트랜스포머 블록 적용
    x = transformer_block(embedding_layer, emb_dim, num_heads, ff_dim, dropout=dropout_rate)

    # Global Average Pooling
    x = GlobalAveragePooling1D()(x)

    # Dropout (최종 분류 전에 추가)
    x = Dropout(dropout_rate)(x)

    # 분류를 위한 Dense 레이어 (num_classes = 56)
    output_layer = Dense(num_classes, activation='softmax')(x)

    # 모델 정의
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

In [22]:
model = build_model(n_most_common_words=80000,
                    emb_dim=128,
                    max_len=130,
                    num_heads=8,
                    ff_dim=128,
                    num_classes=56,
                    dropout_rate=0.3)

In [23]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 130)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 130, 128)       │     10,240,000 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 130, 128)       │        527,488 │ embedding_1[0][0],     │
│ (MultiHeadAttention)      │                        │                │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 130, 128)       │              0 │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 130, 128)       │              0 │ embedding_1[0][0],     │
│                           │                        │                │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 130, 128)       │            256 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 130, 128)       │         16,512 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 130, 128)       │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 130, 128)       │         16,512 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 130, 128)       │              0 │ layer_normalization[0… │
│                           │                        │                │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 130, 128)       │            256 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 128)            │              0 │ layer_normalization_1… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 128)            │              0 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 56)             │          7,224 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 10,808,248 (41.23 MB)

 Trainable params: 10,808,248 (41.23 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# wandb.init(
#     # W&B 프로젝트 이름을 설정
#     project="article classification",

#     # 하이퍼파라미터 및 실행 메타데이터를 추적
#     config={
#         "learning_rate": 0.001,
#         "architecture": "Transformer",
#         "dataset": "titles",
#         "epochs": 5
#         "max_sequence_length": 130,
#         "embedding_dim": 128,
#         "num_heads": 8,
#         "feedforward_dim": 128,
#         "num_classes": 56
#     }
# )

# """
# # 기본 설정
# n_most_common_words = 80000  # 어휘 수 80,000개로 설정
# emb_dim = 128  # 임베딩 벡터의 차원 설정
# max_len = X.shape[1]  # 시퀀스의 길이 (기사 제목의 길이)
# num_heads = 8  # Multi-head attention에서 헤드의 개수
# ff_dim = 128  # Feed Forward Network의 출력 차원
# num_classes = 56  # 최종 클래스의 개수
# """

In [25]:
import matplotlib.pyplot as plt

# 학습 결과 시각화
def plot_training_history(history):
    # 정확도 그래프
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # 손실 그래프
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25)

In [27]:
#컴파일 설정
optimizer=AdamW(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
early_stopping = EarlyStopping(
    monitor='val_loss',     # 검증 손실을 모니터링
    mode='min',             # 'min' 모드로 설정하여, 손실이 감소하는 것을 추적
    patience=5,             # 손실이 더 이상 개선되지 않는 즉시 학습 중단
    restore_best_weights=True  # 가장 성능이 좋았던 가중치로 복원
)

In [29]:
def reset_weights(model):
    # 모델의 모든 레이어의 가중치를 초기화
    for layer in model.layers:
        if hasattr(layer, 'kernel_initializer') and layer.kernel_initializer:
            layer.kernel.assign(layer.kernel_initializer(layer.kernel.shape))
        if hasattr(layer, 'bias_initializer') and layer.bias_initializer:
            layer.bias.assign(layer.bias_initializer(layer.bias.shape))

In [30]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

In [ ]:
epochs=[5,10,15]
learning_rates=[0.0001,0.0003,0.0005]
batch_sizes=[64,128,256]
dropout_rates=[0.3,0.5,0.75]

best_val_loss = np.inf  # 현재 최적의 검증 손실 값
best_val_acc=0
best_hyperparams = {}

for epoch in epochs:
  for learning_rate in learning_rates:
    for batch_size in batch_sizes:
      for dropout_rate in dropout_rates:

        print(f"Recent epoch: {epoch}")
        print(f"Recent learning_rate: {learning_rate}")
        print(f"Recent batch_size: {batch_size}")
        print(f"Recent dropout_rate: {dropout_rate}")

        model = build_model(n_most_common_words=n_most_common_words,
                    emb_dim=emb_dim,
                    max_len=max_len,
                    num_heads=num_heads,
                    ff_dim=ff_dim,
                    num_classes=num_classes,
                    dropout_rate=dropout_rate)

        reset_weights(model)

        optimizer=AdamW(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        history = model.fit(
        X_train,
        y_train,
        epochs=epoch,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=[early_stopping])


        val_loss = min(history.history['val_loss'])
        val_acc = max(history.history['val_accuracy'])
        if val_loss < best_val_loss and val_acc > best_val_acc:
                    best_val_loss = val_loss
                    best_val_acc = val_acc
                    best_hyperparams['epoch'] = epoch
                    best_hyperparams['learning_rate'] = learning_rate
                    best_hyperparams['batch_size'] = batch_size
                    best_hyperparams['dropout_rate'] = dropout_rate

        #plot_training_history(history)

# 최종 결과 출력
print(f"Best val_loss: {best_val_loss}")
print(f"Best val_acc: {best_val_acc}")
print(f"Best Hyperparameters: Epoch: {best_hyperparams['epoch']}, "
      f"Learning Rate: {best_hyperparams['learning_rate']}, "
      f"Batch Size: {best_hyperparams['batch_size']}, "
      f"Dropout Rate: {best_hyperparams['dropout_rate']}")

Recent epoch: 5
Recent learning_rate: 0.0001
Recent batch_size: 64
Recent dropout_rate: 0.3
Epoch 1/5
823/823 ━━━━━━━━━━━━━━━━━━━━ 48s 39ms/step - accuracy: 0.2977 - loss: 3.6847 - val_accuracy: 0.3070 - val_loss: 6.0568
Epoch 2/5
823/823 ━━━━━━━━━━━━━━━━━━━━ 62s 28ms/step - accuracy: 0.3064 - loss: 3.4646 - val_accuracy: 0.3160 - val_loss: 4.6470
Epoch 3/5
823/823 ━━━━━━━━━━━━━━━━━━━━ 42s 29ms/step - accuracy: 0.4792 - loss: 2.4592 - val_accuracy: 0.7344 - val_loss: 1.2244
Epoch 4/5
823/823 ━━━━━━━━━━━━━━━━━━━━ 40s 28ms/step - accuracy: 0.7952 - loss: 0.9361 - val_accuracy: 0.7778 - val_loss: 1.0135
Epoch 5/5
823/823 ━━━━━━━━━━━━━━━━━━━━ 42s 29ms/step - accuracy: 0.8563 - loss: 0.6318 - val_accuracy: 0.7959 - val_loss: 0.9841
Recent epoch: 5
Recent learning_rate: 0.0001
Recent batch_size: 64
Recent dropout_rate: 0.5
Epoch 1/5
823/823 ━━━━━━━━━━━━━━━━━━━━ 41s 39ms/step - accuracy: 0.2950 - loss: 3.6733 - val_accuracy: 0.3070 - val_loss: 7.3635
Epoch 2/5
823/823 ━━━━━━━━━━━━━━━━━━━━ 28s

In [40]:
# # 최적화된 하이퍼파라미터
# learning_rate = 0.0005        # 학습률 (조정 가능)
# batch_size = 128             # 배치 크기
# epochs = 10                  # 최대 에포크 수
# dropout_rate = 0.8           # 드롭아웃 비율
# l2_lambda = 0.001             # L2 정규화 값
# patience = 7                  # 조기 종료를 위한 patience
# min_delta = 0.001             # 조기 종료를 위한 최소 손실 감소량
# validation_split = 0.2        # 검증 데이터 비율


In [41]:
# optimizer=AdamW(learning_rate=learning_rate)
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
# history = model.fit(
#     X_train,
#     y_train,
#     epochs=epochs,
#     batch_size=batch_size,
#     validation_split=validation_split,
#     callbacks=[early_stopping])

Epoch 1/10
412/412 ━━━━━━━━━━━━━━━━━━━━ 39s 75ms/step - accuracy: 0.3071 - loss: 3.5400 - val_accuracy: 0.6488 - val_loss: 1.5889
Epoch 2/10
412/412 ━━━━━━━━━━━━━━━━━━━━ 30s 57ms/step - accuracy: 0.7400 - loss: 1.1682 - val_accuracy: 0.7841 - val_loss: 0.9442
Epoch 3/10
412/412 ━━━━━━━━━━━━━━━━━━━━ 42s 59ms/step - accuracy: 0.8680 - loss: 0.5267 - val_accuracy: 0.8024 - val_loss: 0.9384
Epoch 4/10
412/412 ━━━━━━━━━━━━━━━━━━━━ 23s 56ms/step - accuracy: 0.9115 - loss: 0.3406 - val_accuracy: 0.8087 - val_loss: 0.9881
Epoch 5/10


KeyboardInterrupt: 

In [ ]:
# plot_training_history(history)

In [ ]:
# 모델 전체를 저장하는 코드 (모델 아키텍처, 가중치, 컴파일 정보 포함)
model.save('/content/drive/MyDrive/코딩공부/dacon_project/result/bbc_edit_model.h5')  # 모델을 'model.h5' 파일로 저장

print("모델이 'bbc_edit_model.h5'로 저장되었습니다.")


In [ ]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train.csv')
# category_counts = df['분류'].value_counts()

# # 결과를 출력합니다.
# print(category_counts[:40])

In [ ]:
txt = ["용인특례시, 가을 신학기 학교 급식 공급업체 위생점검"]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
labels = [
    "지역", "경제:부동산", "사회:사건_사고", "경제:반도체", "사회:사회일반", "사회:교육_시험",
    "정치:국회_정당", "사회:의료_건강", "경제:취업_창업", "스포츠:올림픽_아시안게임",
    "경제:산업_기업", "문화:전시_공연", "경제:자동차", "경제:경제일반", "사회:장애인",
    "스포츠:골프", "정치:선거", "경제:유통", "IT_과학:모바일", "사회:여성",
    "사회:노동_복지", "사회:환경", "경제:서비스_쇼핑", "경제:무역", "정치:행정_자치",
    "국제", "문화:방송_연예", "스포츠:축구", "경제:금융_재테크", "정치:청와대",
    "문화:출판", "IT_과학:IT_과학일반", "IT_과학:인터넷_SNS", "문화:미술_건축",
    "정치:정치일반", "IT_과학:과학", "문화:문화일반", "문화:학술_문화재", "문화:요리_여행",
    "경제:자원", "문화:종교", "IT_과학:콘텐츠", "사회:미디어", "사회:날씨",
    "스포츠:농구_배구", "문화:음악", "문화:생활", "IT_과학:보안", "스포츠:월드컵",
    "경제:증권_증시", "정치:북한", "정치:외교", "스포츠:스포츠일반", "문화:영화",
    "스포츠:야구", "경제:외환"
]
# print(pred, labels[np.argmax(pred)])

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. test.csv 파일 로드
test_df = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/test.csv')

# 2. 데이터 전처리: 제목을 시퀀스로 변환
# 학습 시 사용했던 tokenizer를 사용하여 제목을 시퀀스로 변환
sequences_test = tokenizer.texts_to_sequences(test_df['제목'].values)

# 3. 시퀀스의 길이를 동일하게 맞추기 (max_len에 맞춰 패딩)
X_test = pad_sequences(sequences_test, maxlen=max_len)

# 4. 모델을 사용하여 예측
predictions = model.predict(X_test)

# 5. 예측 결과에서 가장 확률이 높은 클래스를 추출
predicted_labels = predictions.argmax(axis=1)
predicted_labels[predicted_labels>45]=0

predicted_class_names=[categories[label] for label in predicted_labels]

# 6. submission 데이터프레임 생성 (id와 예측된 레이블을 저장)
submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    '분류': predicted_class_names
})

# 7. submission.csv 파일로 저장
submission_df.to_csv('/content/drive/MyDrive/코딩공부/dacon_project/result/submission.csv', index=False)

print("Submission 파일이 생성되었습니다.")


In [ ]:
df_test=pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/test.csv')
df_result=pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/result/submission.csv')
print("Shpe of test Data",df_test.shape)
print("Shpe of test Data",df_result.shape)
# df_test.head(30)

In [ ]:
df_result.head(30)

In [ ]:
print(df_result['분류'].value_counts())


In [ ]:
print(df_result['분류'])